## Supervised

In [1]:
# BERT model and classification head are trainable, and updated jointly (learn from one another)
# Fine-tuning a pretrained BERT model for sentiment anaylsis
from datasets import load_dataset

tomatoes = load_dataset("rotten_tomatoes")
train_data, test_data = tomatoes["train"], tomatoes["test"]

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)

In [17]:
# logged metrics, detects overfitting
import numpy as np
from datasets import load_metric
from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    predictions = np.argmax(logits, axis=1)
    f1 = f1_score(labels, predictions, average="binary")
    
    #load_f1 = load_metric("f1")
    #f1 = load_f1.compute(predictions=predictions, references=labels, average="binary")["f1"]
    return {"f1": f1}

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_33148/1447919989.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.evaluate()

{'eval_loss': 0.8463835716247559,
 'eval_model_preparation_time': 0.0182,
 'eval_f1': 0.6666666666666666,
 'eval_runtime': 5.6318,
 'eval_samples_per_second': 189.282,
 'eval_steps_per_second': 11.897}

### Freezing layers

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# check what we might want to freeze
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [22]:
# freeze everything except classification head
for name, param in model.named_parameters():
    if name.startswith("classifier"):
        param.requires_grad = True
    else:
        param.requires_grad = False

In [23]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_33148/1482939272.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.686100


TrainOutput(global_step=534, training_loss=0.6863191511747095, metrics={'train_runtime': 51.1776, 'train_samples_per_second': 166.675, 'train_steps_per_second': 10.434, 'total_flos': 227605451772240.0, 'train_loss': 0.6863191511747095, 'epoch': 1.0})

## ^ training is much faster (only training the classification head)

In [24]:
trainer.evaluate()

{'eval_loss': 0.6762992143630981,
 'eval_f1': 0.6201966041108132,
 'eval_runtime': 6.2526,
 'eval_samples_per_second': 170.489,
 'eval_steps_per_second': 10.716,
 'epoch': 1.0}

### notice f1 score is low (0.62) lets freeze only the first 10 encoders

In [26]:
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# encoder block 11 starts at index 165 freeze everything before
for index, (name, param) in enumerate(model.named_parameters()):
    if index < 165:
        param.requires_grad = False

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_33148/757739267.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.476800


TrainOutput(global_step=534, training_loss=0.472287653090802, metrics={'train_runtime': 68.7508, 'train_samples_per_second': 124.071, 'train_steps_per_second': 7.767, 'total_flos': 227605451772240.0, 'train_loss': 0.472287653090802, 'epoch': 1.0})

In [29]:
trainer.evaluate()

{'eval_loss': 0.4116963744163513,
 'eval_f1': 0.8164435946462715,
 'eval_runtime': 5.3717,
 'eval_samples_per_second': 198.449,
 'eval_steps_per_second': 12.473,
 'epoch': 1.0}

### ^ much better F1 score

## Few-shot classification

In [31]:
!pip install setfit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [32]:
from setfit import sample_dataset

sampled_train_data = sample_dataset(tomatoes["train"], num_samples=16)

In [33]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [34]:
# contrastive learning
from setfit import TrainingArguments as SetFitTrainingArguments
from setfit import Trainer as SetFitTrainer

args = SetFitTrainingArguments(
    num_epochs=3,
    num_iterations=20 # text pairs
)
args.eval_strategy = args.evaluation_strategy

trainer = SetFitTrainer(
    model=model,
    args=args,
    train_dataset=sampled_train_data,
    eval_dataset=test_data,
    metric="f1"
)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [35]:
trainer.train()

***** Running training *****
  Num unique pairs = 1280
  Batch size = 16
  Num epochs = 3


Step,Training Loss,Validation Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

### 1280 because 20 * 32 = 680 and 680 * 2 = 1280 sentence pairs from 32 labeled sentences!

### Specify a classification head

In [ ]:
# model = SetFitModel.from_pretrained(
#     "sentence-transformers/all-mpnet-base-v2",
#     use_differentiable_head=True,
#     head_params={"out_features": num_classes}, # classes to predict
# )
# trainer = SetFitTrainer(model=model, ...)

In [36]:
trainer.evaluate()

***** Running evaluation *****


{'f1': 0.8519230769230769}

## ^ Even better F1 score! On only 32 labeled documents!

## In order to train on a specific domain, 3 step process:
- use pretrain model
- continue the training of the pretrain model with domain specific data
- classification task

In [7]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_train = tokenized_train.remove_columns("label")
tokenized_test = test_data.map(preprocess_function, batched=True)
tokenized_test = tokenized_test.remove_columns("label")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

In [38]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15 # 15% a token is masked
)

In [40]:
training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator
)

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_33148/3385921974.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [42]:
tokenizer.save_pretrained("mlm")
trainer.train()
model.save_pretrained("mlm")

Step,Training Loss
500,2.602900
1000,2.377200
1500,2.299300
2000,2.189000
2500,2.139100
3000,2.094700
3500,2.058600
4000,1.988500
4500,1.978700
5000,1.966500


In [57]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model="bert-base-cased")
preds = mask_filler("What a horrible [MASK]!")

for pred in preds:
    print(f">>> {pred['sequence']}")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


>>> What a horrible idea!
>>> What a horrible dream!
>>> What a horrible thing!
>>> What a horrible day!
>>> What a horrible thought!


In [55]:
mask_filler = pipeline("fill-mask", model="mlm")
preds = mask_filler("What a horrible [MASK]!")

for pred in preds:
    print(f">>> {pred['sequence']}")

Device set to use mps:0


>>> What a horrible movie!
>>> What a horrible film!
>>> What a horrible mess!
>>> What a horrible story!
>>> What a horrible comedy!


## ^ biased with the model we fed it

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("mlm", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("mlm")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mlm and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Named-Entity Recognition (NER)
(makes predicitions for individual tokens in a sequence)

In [4]:
dataset = load_dataset("conll2003", trust_remote_code=True)
# lots of different datasets to choose from this one contains
# different types of named entities (person, organization, location, misc, no entity)

# Limit the training data to the first 1000 samples
# train_subset = dataset["train"].select(range(1000))

# Create a new dataset with the limited training data
# dataset = dataset.with_format("torch")  # Adjust format if necessary
# dataset["train"] = train_subset

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
example = dataset["train"][401]
example

{'id': '401',
 'tokens': ['Karol',
  'Kucera',
  '(',
  'Slovakia',
  ')',
  'beat',
  'Hicham',
  'Arazi',
  '(',
  'Morocco',
  ')',
  '7-6',
  '(',
  '7-4',
  ')'],
 'pos_tags': [22, 22, 4, 22, 5, 37, 22, 22, 4, 22, 5, 11, 4, 11, 5],
 'chunk_tags': [11, 12, 0, 11, 0, 21, 11, 12, 0, 11, 0, 3, 0, 11, 0],
 'ner_tags': [1, 2, 0, 5, 0, 0, 1, 2, 0, 5, 0, 0, 0, 0, 0]}

### labels ^ ner_tags key 

In [6]:
# mapping person (PER), orginization (ORG), location (LOC) etc
# B (beginning) or I (indside) two tokens are part of the same phrase, start of phrase B, folowed by I
# example: first name (B) last name (I) was at location (B)
label2id = {
    "0": 0, "B-PER": 1, "I-PER": 2, "B-ORG": 3, "I-ORG": 4,
    "B-LOC": 5, "I-LOC": 6, "B-MISC": 7, "I-MISC": 8
}
id2label = {index: label for label, index in label2id.items()}
label2id

{'0': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [7]:
from transformers import AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
token_ids = tokenizer(example["tokens"], is_split_into_words=True)["input_ids"]
sub_tokens = tokenizer.convert_ids_to_tokens(token_ids)
sub_tokens

['[CLS]',
 'Ka',
 '##rol',
 'Ku',
 '##cer',
 '##a',
 '(',
 'Slovakia',
 ')',
 'beat',
 'Hi',
 '##cha',
 '##m',
 'Ara',
 '##zi',
 '(',
 'Morocco',
 ')',
 '7',
 '-',
 '6',
 '(',
 '7',
 '-',
 '4',
 ')',
 '[SEP]']

In [11]:
# We need to make sure to properly label not words themselves but the tokens so that B can be followed by I if separated
# we use -100 for special tokens ([SEP], [CLS])
def align_labels(examples):
    token_ids = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True
    )
    labels = examples["ner_tags"]

    updated_labels = []
    for index, label in enumerate(labels):
        word_ids = token_ids.word_ids(batch_index=index)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx != previous_word_idx:
                previous_word_idx = word_idx
                label_ids.append(-100 if word_idx is None else label[word_idx])
            elif word_idx is None:
                label_ids.append(-100)
            else:
                # label_ids.append(label[word_idx])
                updated_label = label[word_idx]
                if updated_label % 2 == 1:
                    updated_label += 1
                label_ids.append(updated_label)
        updated_labels.append(label_ids)

    token_ids["labels"] = updated_labels
    return token_ids

tokenized = dataset.map(align_labels, batched=True, batch_size=10)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [12]:
print(f"Original: {example['ner_tags']}")
print(f"Updated: {tokenized['train'][401]['labels']}")

Original: [1, 2, 0, 5, 0, 0, 1, 2, 0, 5, 0, 0, 0, 0, 0]
Updated: [-100, 1, 2, 2, 2, 2, 0, 5, 0, 0, 1, 2, 2, 2, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [16]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Installing backend dependencies ... one
  Preparing metadata (pyproject.toml) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16217 sha256=be8c04594381af5ffdc6be46950b664f6a7d5820d7e8aaf2f01a032134772bff
  Stored in directory: /Users/mathias/Library/Caches/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [28]:
# evaluate performance
import evaluate
import numpy as np
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)  # Shape: (num_samples, seq_length)

    true_predictions = []
    true_labels = []

    # Process each sequence in the batch
    for prediction, label in zip(predictions, labels):
        filtered_predictions = []
        filtered_labels = []
        for p, l in zip(prediction, label):
            if l != -100:  # Ignore padding tokens
                filtered_predictions.append(id2label[p])
                filtered_labels.append(id2label[l])
        true_predictions.append(filtered_predictions)  # Append the sequence
        true_labels.append(filtered_labels)  # Append the sequence

    # Compute metrics using seqeval
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [30]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    "model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.train()

/var/folders/dt/x2hsbqvj7bsd0klwc9f8ff8m0000gn/T/ipykernel_41640/2095477416.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.050100


TrainOutput(global_step=878, training_loss=0.0476915733146233, metrics={'train_runtime': 265.1951, 'train_samples_per_second': 52.946, 'train_steps_per_second': 3.311, 'total_flos': 351240792638148.0, 'train_loss': 0.0476915733146233, 'epoch': 1.0})

In [31]:
trainer.evaluate()

/Users/mathias/Documents/research2025/LLMs/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eval_loss': 0.1686190813779831,
 'eval_f1': 0.9085269741190827,
 'eval_accuracy': 0.9704542947637131,
 'eval_runtime': 15.807,
 'eval_samples_per_second': 218.448,
 'eval_steps_per_second': 13.665,
 'epoch': 1.0}

In [32]:
# saving model to use for pipeline inference, allows us to manually check to see if satisfied with output
from transformers import pipeline

trainer.save_model("ner_model")

# run inference
token_classifier = pipeline(
    "token-classification",
    model="ner_model",
)
token_classifier("My name is Mathias, nice to meet you. Lets go to the Arcade")

Device set to use mps:0


[{'entity': '0',
  'score': 0.999423,
  'index': 1,
  'word': 'My',
  'start': 0,
  'end': 2},
 {'entity': '0',
  'score': 0.9991417,
  'index': 2,
  'word': 'name',
  'start': 3,
  'end': 7},
 {'entity': '0',
  'score': 0.9992617,
  'index': 3,
  'word': 'is',
  'start': 8,
  'end': 10},
 {'entity': 'B-PER',
  'score': 0.9910926,
  'index': 4,
  'word': 'Mathias',
  'start': 11,
  'end': 18},
 {'entity': '0',
  'score': 0.99933213,
  'index': 5,
  'word': ',',
  'start': 18,
  'end': 19},
 {'entity': '0',
  'score': 0.9991381,
  'index': 6,
  'word': 'nice',
  'start': 20,
  'end': 24},
 {'entity': '0',
  'score': 0.9992908,
  'index': 7,
  'word': 'to',
  'start': 25,
  'end': 27},
 {'entity': '0',
  'score': 0.9992685,
  'index': 8,
  'word': 'meet',
  'start': 28,
  'end': 32},
 {'entity': '0',
  'score': 0.9993894,
  'index': 9,
  'word': 'you',
  'start': 33,
  'end': 36},
 {'entity': '0',
  'score': 0.99974746,
  'index': 10,
  'word': '.',
  'start': 36,
  'end': 37},
 {'entity

### correctly identified a person!